In [4]:
!mkdir ckpt2

mkdir: cannot create directory ‘ckpt2’: File exists


In [ ]:
# 1. Thêm các thư viện cần thiết và function
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab.patches import cv2_imshow 
import cv2 as cv
import random
import time
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.datasets import mnist

#3. Load dataset
  
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

#4. Thêm nhiễu
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

def train_model(input_shape=(28,28,1)):
    input_img = Input(input_shape))  # adapt this if using `channels_first` image data format
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same', name='encoder')(x)

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(inputs = input_img,outputs = decoded)
    autoencoder.compile(optimizer='mse', loss='adam')
    
    path1 = 'ckpt2/best.hdf5' 
    callback = [ModelCheckpoint('best_model.h5', 
                                verbose=1, 
                                save_best_only=True,
                                save_weights_only=True)]
  
    
    autoencoder.fit(x_train_noisy, x_train,
                    epochs=10,
                    batch_size=128,
                    shuffle=True,
                    validation_data=(x_test_noisy, x_test),
                    callbacks=callback,
                    verbose=1)

    autoencoder.save('autoencoder.h5')

train_model()

